In [2]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "af4a46a8-f352-4e18-8b59-66402d2c80b9",
   "metadata": {},
   "source": [
    "# Steps 6 - 8"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 64,
   "id": "3a41aa86-8f12-4355-92a3-742756da66bb",
   "metadata": {
    "scrolled": true
   },
   "outputs": [
    {
     "ename": "MemoryError",
     "evalue": "Unable to allocate 469. MiB for an array with shape (8582, 14339) and data type float32",
     "output_type": "error",
     "traceback": [
      "\u001b[31m---------------------------------------------------------------------------\u001b[39m",
      "\u001b[31mMemoryError\u001b[39m                               Traceback (most recent call last)",
      "\u001b[36mCell\u001b[39m\u001b[36m \u001b[39m\u001b[32mIn[64]\u001b[39m\u001b[32m, line 8\u001b[39m\n\u001b[32m      6\u001b[39m \u001b[38;5;66;03m# Load JWST image\u001b[39;00m\n\u001b[32m      7\u001b[39m \u001b[38;5;28;01mwith\u001b[39;00m fits.open(\u001b[33m\"\u001b[39m\u001b[33mjw02731-o001_t017_nircam_clear-f090w_i2d.fits\u001b[39m\u001b[33m\"\u001b[39m) \u001b[38;5;28;01mas\u001b[39;00m hdul:\n\u001b[32m----> \u001b[39m\u001b[32m8\u001b[39m     data = \u001b[43mhdul\u001b[49m\u001b[43m[\u001b[49m\u001b[32;43m1\u001b[39;49m\u001b[43m]\u001b[49m\u001b[43m.\u001b[49m\u001b[43mdata\u001b[49m\u001b[43m.\u001b[49m\u001b[43mastype\u001b[49m\u001b[43m(\u001b[49m\u001b[43mnp\u001b[49m\u001b[43m.\u001b[49m\u001b[43mfloat32\u001b[49m\u001b[43m)\u001b[49m\n",
      "\u001b[31mMemoryError\u001b[39m: Unable to allocate 469. MiB for an array with shape (8582, 14339) and data type float32"
     ]
    }
   ],
   "source": [
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import sep\n",
    "from astropy.io import fits\n",
    "from astropy.wcs import WCS\n",
    "from reproject import reproject_interp\n",
    "from astropy.visulization import simple_norm\n",
    "from gc # For memory cleanup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f85466b5-2366-4e8b-870b-4ba64314fb1b",
   "metadata": {},
   "outputs": [],
   "source": [
    "def downsample(data, factor):\n",
    "    shape = (data.shape[0] // factor, factor,\n",
    "             data.shape[1] // factor, factor)\n",
    "    return data[:shape[0]*factor, :shape[2]*factor].reshape(shape).mean(-1).mean(1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "c0486aff-40df-4402-a760-084d6eeb7e0a",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load JWST image\n",
    "with fits.open(\"jw02731-o001_t017_nircam_clear-f090w_i2d.fits\", memmap=True) as hdul:\n",
    "    data = hdul[1].data.astype(np.float32)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "678cb8e4-2bc7-4aab-8861-07b70fc885bd",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Downsample before processing to save memory\n",
    "down_data = downsample(data, 4)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ddee62e3-5d99-44dd-87ea-d1326382f1d4",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Subtract background\n",
    "bkg = sep.Background(down_data)\n",
    "data_sub = down_data - bkg.back()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "173d1fcd-0d80-4b52-b137-f968de17de9f",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Estimate and subtract background\n",
    "small_data = data[:1000, :1000]\n",
    "bkg = sep.Background(data)\n",
    "small_data_sub = small_data - bkg.back()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "18fe3067-3a3e-4545-9060-0aeb001ed590",
   "metadata": {},
   "outputs": [],
   "source": [
    "def downsample(data, factor):\n",
    "    shape = (data.shape[0] // factor, factor,\n",
    "             data.shape[1] // factor, factor)\n",
    "    return data[:shape[0]*factor, :shape[2]*factor].reshape(shape).mean(-1).mean(1)\n",
    "\n",
    "# Apply downsampling\n",
    "down_data = downsample(data_sub, 4)  # Try factor=4 or 8 to test memory\n",
    "\n",
    "# Recompute background on smaller image\n",
    "bkg = sep.Background(down_data)\n",
    "down_data_sub = down_data - bkg.back()\n",
    "\n",
    "# Try extraction again\n",
    "sep.set_extract_pixstack(1000000)  # You already added this\n",
    "objects = sep.extract(down_data_sub, thresh=5.0, err=bkg.globalrms)\n",
    "print(\"Number of sources detected:\", len(objects))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "204bf410-2a98-482e-ad37-ffa61b58a38c",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get fluxes and plot histogram\n",
    "fluxes = np.array([obj['flux'] for obj in objects])\n",
    "plt.hist(fluxes, bins=50, log=True)\n",
    "plt.title(\"Flux Distribution of Detected Sources\")\n",
    "plt.xlabel(\"Flux\")\n",
    "plt.ylabel(\"Number of Sources\")\n",
    "plt.savefig(\"step6_flux_histogram.png\", dpi=300)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "1280c8de-331f-4568-8dc2-189840c9daf7",
   "metadata": {},
   "outputs": [],
   "source": [
    "fluxes = np.array([obj['flux'] for obj in objects])\n",
    "plt.hist(fluxes, bins=50, color='skyblue', edgecolor='black')\n",
    "plt.xlabel(\"Flux\")\n",
    "plt.ylabel(\"Number of Sources\")\n",
    "plt.title(\"Histogram of Source Fluxes\")\n",
    "plt.grid(True)\n",
    "plt.savefig(\"step6_flux_histogram.png\")  # Save PNG\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "596325f1-da2a-4989-8d25-67b4efef2b4c",
   "metadata": {},
   "source": [
    "#Step 7"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "3e69704b-4b50-4b03-85dc-8ea1710ee215",
   "metadata": {},
   "outputs": [],
   "source": [
    "mean_flux = np.mean(fluxes)\n",
    "median_flux = np.median(fluxes)\n",
    "std_flux = np.std(fluxes)\n",
    "\n",
    "print(f\"Mean flux: {mean_flux:.2f}\")\n",
    "print(f\"Median flux: {median_flux:.2f}\")\n",
    "print(f\"Standard deviation: {std_flux:.2f}\")\n",
    "\n",
    "# Find the brightest (max) flux source\n",
    "max_flux = np.max(fluxes)\n",
    "max_index = np.argmax(fluxes)\n",
    "max_obj = objects[max_index]\n",
    "x, y = max_obj['x'], max_obj['y']\n",
    "sigma_away = (max_flux - mean_flux) / std_flux\n",
    "\n",
    "print(f\"Largest outlier flux: {max_flux:.2f}\")\n",
    "print(f\"Location (x, y): ({x:.1f}, {y:.1f})\")\n",
    "print(f\"Deviation from mean: {sigma_away:.2f} σ\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "f09ec6c4-f2e1-4143-b0ce-71fbd8633e48",
   "metadata": {},
   "source": [
    "#Step 8"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ed709f66-e770-46cc-93ca-3a2fb679de73",
   "metadata": {},
   "outputs": [],
   "source": [
    "from astropy.io import fits\n",
    "from astropy.wcs import WCS\n",
    "from reproject import reproject_interp"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "07e4d915-d683-4dd4-ace0-706b5e2ec678",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load reference image (f090w) WCS\n",
    "ref_hdul = fits.open(\"jw02731-o001_t017_nircam_clear-f090w_i2d.fits\")\n",
    "ref_header = ref_hdul[1].header\n",
    "ref_wcs = ref_hdul[1]\n",
    "ref_hdul.close()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "0701f6c1-c9c6-4856-972e-8474fb6ea0e3",
   "metadata": {},
   "outputs": [],
   "source": [
    "def downsample(data, factor):\n",
    "    shape = (data.shape[0] // factor, factor,\n",
    "             data.shape[1] // factor, factor)\n",
    "    return data[:shape[0]*factor, :shape[2]*factor].reshape(shape).mean(-1).mean(1)\n",
    "\n",
    "def reproject_to_f090w(file, factor=4):  # Reduce resolution 4x\n",
    "    with fits.open(file) as hdul:\n",
    "        data = hdul[1].data\n",
    "        header = hdul[1].header\n",
    "\n",
    "    # ↓↓↓ Downsample the image\n",
    "    small_data = downsample(data, factor)\n",
    "    small_header = header.copy()\n",
    "\n",
    "    # Important: Adjust WCS resolution to match downsampling\n",
    "    small_header['CDELT1'] *= factor\n",
    "    small_header['CDELT2'] *= factor\n",
    "\n",
    "    # ↓↓↓ Now safe to reproject\n",
    "    reproj, _ = reproject_interp((small_data, small_header), ref_header)\n",
    "    return reproj"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f3680880-d51f-4701-a94a-b8347d69d7dc",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Reproject selected bands\n",
    "img_r = reproject_to_f090w(\"jw02731-o001_t017_nircam_clear-f444w_i2d.fits\")  # Red\n",
    "img_g = reproject_to_f090w(\"jw02731-o001_t017_nircam_clear-f277w_i2d.fits\")  # Green\n",
    "img_b = reproject_to_f090w(\"jw02731-o001_t017_nircam_clear-f090w_i2d.fits\")  # Blue"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "73b171e5-3933-47ad-ac80-dab72b220578",
   "metadata": {},
   "outputs": [],
   "source": [
    "from astropy.visualization import simple_norm\n",
    "\n",
    "# Normalize for display\n",
    "norm_r = simple_norm(img_r, 'sqrt', percent=99.5)\n",
    "norm_g = simple_norm(img_g, 'sqrt', percent=99.5)\n",
    "norm_b = simple_norm(img_b, 'sqrt', percent=99.5)\n",
    "\n",
    "# Stack into RGB image\n",
    "rgb = np.dstack([\n",
    "    norm_r(img_r),\n",
    "    norm_g(img_g),\n",
    "    norm_b(img_b)])\n",
    "\n",
    "# Plot\n",
    "plt.figure(figsize=(10, 10))\n",
    "plt.imshow(rgb, origin='lower')\n",
    "plt.axis('off')\n",
    "plt.title(\"JWST NGC 3324 False Color Composite\")\n",
    "plt.savefig(\"ngc3324_false_color_rgb.png\", dpi=300, bbox_inches='tight')\n",
    "plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.0rc2"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


NameError: name 'true' is not defined